In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
df= pd.read_csv('/content/train (1).csv')
df.head(5)

,cap.shape,cap.color,bruises,stalk.color.above.ring,stalk.color.below.ring,population,Id,poisonous
0,k,e,False,w,w,v,6573,1
1,f,e,True,p,w,y,4426,0
2,b,w,False,w,w,s,7018,0
3,k,g,False,w,w,n,5789,0
4,f,n,True,p,g,v,6187,0


In [ ]:
X_train = df

In [ ]:
y_train = df.pop("poisonous")


In [ ]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6499 entries, 0 to 6498
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   cap.shape               6499 non-null   object
 1   cap.color               6499 non-null   object
 2   bruises                 6499 non-null   bool  
 3   stalk.color.above.ring  6499 non-null   object
 4   stalk.color.below.ring  6499 non-null   object
 5   population              6499 non-null   object
 6   Id                      6499 non-null   int64 
dtypes: bool(1), int64(1), object(5)
memory usage: 311.1+ KB


In [ ]:
X_train.isna().sum()

cap.shape                 0
cap.color                 0
bruises                   0
stalk.color.above.ring    0
stalk.color.below.ring    0
population                0
Id                        0
dtype: int64

Train-test split

Before performing a deeper exploration, we will split the dataset into 2 chunks: train and test. We will use the train set to find patterns in the data and create a model. The test set will remain untouched, unseen, unexplored. It will be the "reality check" for our model, it will let us know whether our model is able to generalize.

Here, we're using the function train_test_split to shuffle the observations randomly and create the train and test sets for both the X and the y objects at the same time, ensuring that they are shuffled the same way:

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [ ]:
X_train

,cap.shape,cap.color,bruises,stalk.color.above.ring,stalk.color.below.ring,population,Id
0,k,e,False,w,w,v,6573
1,f,e,True,p,w,y,4426
2,b,w,False,w,w,s,7018
3,k,g,False,w,w,n,5789
4,f,n,True,p,g,v,6187
...,...,...,...,...,...,...,...
6494,f,g,False,b,b,v,7002
6495,k,e,False,p,p,v,7323
6496,f,g,False,n,b,y,6206
6497,f,n,True,w,w,s,4658


Sklearn pipeline + all features

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline

# select categorical and numerical column names
X_cat_columns = X_train.select_dtypes(exclude="number").copy().columns
X_num_columns = X_train.select_dtypes(include="number").copy().columns

# create numerical pipeline, only with the SimpleImputer(strategy="mean")
numeric_pipe = make_pipeline(
    SimpleImputer(strategy="mean"))
 
 # create categorical pipeline, with the SimpleImputer(fill_value="N_A") and the OneHotEncoder
categoric_pipe = make_pipeline(
    SimpleImputer(strategy="constant", fill_value="N_A"),
    OneHotEncoder(handle_unknown='ignore')
)


In [ ]:
from sklearn.compose import ColumnTransformer

preprocessor = ColumnTransformer(
    transformers=[
        ("num_pipe", numeric_pipe, X_num_columns),
        ("cat_pipe", categoric_pipe, X_cat_columns),
    ]
)


In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier

Decision Tree Classifier

In [ ]:
from sklearn.tree import DecisionTreeClassifier
full_pipeline = make_pipeline(preprocessor, 
                              DecisionTreeClassifier())
param_grid = {
    'decisiontreeclassifier__max_depth': range(2, 12),
    'decisiontreeclassifier__min_samples_leaf': range(3, 10, 2),
    'decisiontreeclassifier__min_samples_split': range(3, 40, 5),
    'decisiontreeclassifier__criterion':['gini', 'entropy']
    }
    
from sklearn.model_selection import GridSearchCV
 
search = GridSearchCV(full_pipeline, # you have defined this beforehand
                      param_grid, # your parameter grid
                      cv=5, # the value for K in K-fold Cross Validation
                      scoring='accuracy', # the performance metric to use, 
                      verbose=1) # we want informative outputs during the training process


In [ ]:
search.fit(X_train, y_train)

Fitting 5 folds for each of 640 candidates, totalling 3200 fits


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('columntransformer',
                                        ColumnTransformer(transformers=[('num_pipe',
                                                                         Pipeline(steps=[('simpleimputer',
                                                                                          SimpleImputer())]),
                                                                         Index(['Id'], dtype='object')),
                                                                        ('cat_pipe',
                                                                         Pipeline(steps=[('simpleimputer',
                                                                                          SimpleImputer(fill_value='N_A',
                                                                                                        strategy='constant')),
                                                                        

In [ ]:
d_tree_score=round(search.score(X_train,y_train)*100, 2)
d_tree_score

95.71

Random Forest Classifier

In [ ]:
full_pipeline = make_pipeline(preprocessor, 
                              RandomForestClassifier())
param_grid = {
    'randomforestclassifier__max_depth': range(2, 12)
    }
    
from sklearn.model_selection import GridSearchCV
 
random = GridSearchCV(full_pipeline, # you have defined this beforehand
                      param_grid, # your parameter grid
                      cv=5, # the value for K in K-fold Cross Validation
                      scoring='accuracy', # the performance metric to use, 
                      verbose=1) # we want informative outputs during the training process


In [ ]:
random.fit(X_train, y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('columntransformer',
                                        ColumnTransformer(transformers=[('num_pipe',
                                                                         Pipeline(steps=[('simpleimputer',
                                                                                          SimpleImputer())]),
                                                                         Index(['Id'], dtype='object')),
                                                                        ('cat_pipe',
                                                                         Pipeline(steps=[('simpleimputer',
                                                                                          SimpleImputer(fill_value='N_A',
                                                                                                        strategy='constant')),
                                                                        

In [ ]:
r_forest_score=round(random.score(X_train,y_train)*100, 2)
r_forest_score

96.48

SVC (Support Vector Classifier)

In [ ]:
full_pipeline = make_pipeline(preprocessor, 
                              SVC())
param_grid = {
    }
    
from sklearn.model_selection import GridSearchCV
 
svc = GridSearchCV(full_pipeline, # you have defined this beforehand
                      param_grid, # your parameter grid
                      cv=5, # the value for K in K-fold Cross Validation
                      scoring='accuracy', # the performance metric to use, 
                      verbose=1) # we want informative outputs during the training process


In [ ]:
svc.fit(X_train, y_train)

Fitting 5 folds for each of 1 candidates, totalling 5 fits


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('columntransformer',
                                        ColumnTransformer(transformers=[('num_pipe',
                                                                         Pipeline(steps=[('simpleimputer',
                                                                                          SimpleImputer())]),
                                                                         Index(['Id'], dtype='object')),
                                                                        ('cat_pipe',
                                                                         Pipeline(steps=[('simpleimputer',
                                                                                          SimpleImputer(fill_value='N_A',
                                                                                                        strategy='constant')),
                                                                        

In [ ]:
svc_score=round(svc.score(X_train,y_train)*100, 2)
svc_score

51.42

Gaussian NB

In [ ]:
full_pipeline = make_pipeline(preprocessor, 
                              SVC())
param_grid = {
    }
    
from sklearn.model_selection import GridSearchCV
 
gaussian = GridSearchCV(full_pipeline, # you have defined this beforehand
                      param_grid, # your parameter grid
                      cv=5, # the value for K in K-fold Cross Validation
                      scoring='accuracy', # the performance metric to use, 
                      verbose=1) # we want informative outputs during the training process


In [ ]:
gaussian.fit(X_train, y_train)

Fitting 5 folds for each of 1 candidates, totalling 5 fits


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('columntransformer',
                                        ColumnTransformer(transformers=[('num_pipe',
                                                                         Pipeline(steps=[('simpleimputer',
                                                                                          SimpleImputer())]),
                                                                         Index(['Id'], dtype='object')),
                                                                        ('cat_pipe',
                                                                         Pipeline(steps=[('simpleimputer',
                                                                                          SimpleImputer(fill_value='N_A',
                                                                                                        strategy='constant')),
                                                                        

In [ ]:
gaussian_score=round(gaussian.score(X_train,y_train)*100, 2)
gaussian_score

51.42

In [ ]:
test_file= pd.read_csv('/content/test.csv')

In [ ]:
test_file= test_file.loc[:, ~test_file.columns.str.contains('^Unnamed')]

In [ ]:
random.predict(test_file)

array([1, 1, 0, ..., 1, 0, 1])

In [ ]:
result =random.predict(test_file)
test_file["poisonous"] = result
test_file = test_file[["Id","poisonous"]]
#test_file = test_file.reset_index()

In [ ]:
test_file

,Id,poisonous
0,5165,1
1,4281,1
2,231,0
3,3890,0
4,1521,1
...,...,...
1620,983,1
1621,7055,0
1622,7456,1
1623,5132,0


In [ ]:
test_file.to_csv("mushroom_forest.csv", index = False)

from google.colab import files
files.download("mushroom_forest.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Decision tree gave me a good result with good accuracy